In [ ]:
!pip install langchain faiss-gpu modelscope sentence_transformers

In [2]:
import os

import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [5]:
# 模型下载
from modelscope import snapshot_download

model_dir = snapshot_download('Jerry0/m3e-base')
# model_dir
embeddings = HuggingFaceEmbeddings(
                model_name=model_dir,
                model_kwargs={'device': 'cuda'})

In [6]:
# modelscope 下载的模型地址
model_dir

'/root/.cache/modelscope/hub/Jerry0/m3e-base'

In [8]:
data = {
    'text': ['湖北省十四五规划有哪些功能定位?', '湖北省十四五规划中功能定位有多少个?',
             '湖北省十四五规划有哪些基本遵循?', '湖北省十四五规划中基本遵循有多少个?',
             '湖北省十四五规划有哪些重要项目?']
    }
df = pd.DataFrame(data)

## 如果有现成的csv，直接导入就行
# csv_file = 'data/qa_pairs.csv'
# df = pd.read_csv(csv_file)

loader = DataFrameLoader(df, page_content_column="text")
data = loader.load()

In [9]:
data

[Document(page_content='湖北省十四五规划有哪些功能定位?'),
 Document(page_content='湖北省十四五规划中功能定位有多少个?'),
 Document(page_content='湖北省十四五规划有哪些基本遵循?'),
 Document(page_content='湖北省十四五规划中基本遵循有多少个?'),
 Document(page_content='湖北省十四五规划有哪些重要项目?')]

In [10]:
# 向量库构建
faiss_file = 'db/guihua_kg_faiss_store.faiss'
db = FAISS.from_documents(data, embeddings)
if not os.path.exists(faiss_file):
    db = FAISS.from_documents(data, embeddings)
    db.save_local(faiss_file)
else:
    db = FAISS.load_local(faiss_file, embeddings=embeddings)

In [11]:
db.similarity_search('功能定位', k=3)

[Document(page_content='湖北省十四五规划中功能定位有多少个?'),
 Document(page_content='湖北省十四五规划有哪些功能定位?'),
 Document(page_content='湖北省十四五规划有哪些重要项目?')]